In [52]:
import pandas as pd
import numpy as np
import pulp

In [53]:
game_weeks_to_consider = ['GW01','GW02','GW03']

In [54]:
player_stats_raw = pd.read_csv('merged_gw.csv')

In [55]:
player_stats_raw['id'] = player_stats_raw['name'].str.replace(' ','_') +'__'+ player_stats_raw['team'].str.replace(' ','_') +'__'+ player_stats_raw['position'].str.replace(' ','_')
player_stats_raw['team_id'] = 'Team__' + player_stats_raw.team.astype(str).str.replace(' ','_')
player_stats_raw['game_week'] = 'GW' + player_stats_raw['GW'].astype(str).str.pad(2,fillchar='0')
player_stats_raw['points'] = player_stats_raw['total_points']

In [56]:
print(player_stats_raw['id'].nunique())
print(player_stats_raw.shape)

581
(1697, 40)


In [57]:
player_stats_raw = player_stats_raw[['game_week','id','points','value','position','minutes','team_id']]
player_stats_raw = player_stats_raw[player_stats_raw['game_week'].isin(game_weeks_to_consider)]
player_stats_raw

,game_week,id,points,value,position,minutes,team_id
0,GW01,Eric_Bailly__Man_Utd__DEF,0,50,DEF,0,Team__Man_Utd
1,GW01,Keinan_Davis__Aston_Villa__FWD,0,45,FWD,0,Team__Aston_Villa
2,GW01,Ayotomiwa_Dele-Bashiru__Watford__MID,0,45,MID,0,Team__Watford
3,GW01,James_Ward-Prowse__Southampton__MID,2,65,MID,90,Team__Southampton
4,GW01,Bruno_Miguel_Borges_Fernandes__Man_Utd__MID,20,120,MID,90,Team__Man_Utd
...,...,...,...,...,...,...,...
1692,GW03,Wilfred_Ndidi__Leicester__MID,2,50,MID,90,Team__Leicester
1693,GW03,Matt_Ritchie__Newcastle__DEF,1,50,DEF,90,Team__Newcastle
1694,GW03,Nathan_Redmond__Southampton__MID,1,59,MID,32,Team__Southampton
1695,GW03,Mathew_Ryan__Brighton__GK,0,45,GK,0,Team__Brighton


In [58]:
all_ids = player_stats_raw[['id']].drop_duplicates().reset_index(drop=True)
all_ids['merge'] = 1

all_game_weeks = player_stats_raw[['game_week']].drop_duplicates().reset_index(drop=True)
all_game_weeks['merge'] = 1

player_stats = pd.merge(all_ids,all_game_weeks,how='inner',on=['merge'])
print('num of player ids :',len(all_ids))
print('num of game_weeks :',len(all_game_weeks))
print('expected num of rows (#ids * #gws) {} : {}'.format(len(all_ids)*len(all_game_weeks), len(player_stats)))

print()
print('merging rest of information')
print(player_stats.shape)
player_stats = player_stats.merge(player_stats_raw,how='left',on=['id','game_week'])
print(player_stats.shape)

# player_stats['data_available'] = np.where(player_stats.isna().any(axis=1),0,1)
# print()
# print('sum of data_available - {}, should be equal to row in player_stats_raw - {}'.format(player_stats['data_available'].sum(), len(player_stats_raw)))

num of player ids : 581
num of game_weeks : 3
expected num of rows (#ids * #gws) 1743 : 1743

merging rest of information
(1743, 3)
(1743, 8)


In [59]:
player_stats = player_stats.sort_values(by=['id','game_week'])

print(player_stats[['points','value','minutes']].sum().sum())
player_stats[['points','value','minutes']] = player_stats[['points','value','minutes']].fillna(0)
print(player_stats[['points','value','minutes']].sum().sum())

print(player_stats[['id','position','team_id']].groupby(by='id').agg('nunique').max())
player_stats[['position','team_id']] = player_stats.groupby('id')[['position','team_id']].apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
print(player_stats[['id','position','team_id']].groupby(by='id').agg('nunique').max())

player_stats

151058.0
151058.0
position    1
team_id     1
dtype: int64
position    1
team_id     1
dtype: int64


,id,merge,game_week,points,value,position,minutes,team_id
1422,Aaron_Connolly__Brighton__FWD,1,GW01,0.0,55.0,FWD,0.0,Team__Brighton
1423,Aaron_Connolly__Brighton__FWD,1,GW02,1.0,55.0,FWD,45.0,Team__Brighton
1424,Aaron_Connolly__Brighton__FWD,1,GW03,0.0,55.0,FWD,0.0,Team__Brighton
1620,Aaron_Cresswell__West_Ham__DEF,1,GW01,7.0,55.0,DEF,90.0,Team__West_Ham
1621,Aaron_Cresswell__West_Ham__DEF,1,GW02,2.0,55.0,DEF,90.0,Team__West_Ham
...,...,...,...,...,...,...,...,...
1699,Álvaro_Fernández__Brentford__GK,1,GW02,0.0,45.0,GK,0.0,Team__Brentford
1700,Álvaro_Fernández__Brentford__GK,1,GW03,0.0,45.0,GK,0.0,Team__Brentford
705,Çaglar_Söyüncü__Leicester__DEF,1,GW01,6.0,50.0,DEF,90.0,Team__Leicester
706,Çaglar_Söyüncü__Leicester__DEF,1,GW02,0.0,50.0,DEF,90.0,Team__Leicester


In [60]:
player_stats.isna().sum().sum()

0

In [61]:
# objective:
#     Max (SUM(is_player_in_team*points))

# contraints:
#     SUM(is_player_in_team*cost + is_player_in_subs*cost)<=1000
#     SUM(is_player_in_team[where element==Def] + is_player_in_subs[where element==Def])==5,..
#     SUM(is_player_in_team[where element==Def])>=3,..
#     a player should not be in_team and in_sub
#     ForEach(is_player_in_team*minutes>=45) --for getting good subs

In [62]:
model = pulp.LpProblem("Constrained_Value_Maximisation", pulp.LpMaximize)

In [63]:
# add cost>0 constraint

In [64]:
is_player_in_team={}
is_player_in_subs={}
is_player_captain={}
is_player_sold={}
is_player_bought={}
points={}
costs={}
position={}
team={}
playerid={}
for gw_i, gw in enumerate(game_weeks_to_consider):      
    gw_stats = player_stats[player_stats['game_week']==gw]
    points[gw] = gw_stats['points'].fillna(0).tolist()
    costs[gw] = gw_stats['value'].fillna(0).tolist()
    position[gw] = gw_stats['position'].fillna(0).tolist()
    team[gw] = gw_stats['team_id'].fillna(0).tolist()
    playerid[gw] = gw_stats['id'].fillna(0).tolist()
    
    is_player_in_team[gw] = [pulp.LpVariable(x+'__'+gw+'__team',lowBound=0, upBound=1, cat='Integer') for x in gw_stats['id']]
    is_player_in_subs[gw] = [pulp.LpVariable(x+'__'+gw+'__subs',lowBound=0, upBound=1, cat='Integer') for x in gw_stats['id']]
    is_player_captain[gw] = [pulp.LpVariable(x+'__'+gw+'__capt',lowBound=0, upBound=1, cat='Integer') for x in gw_stats['id']]
    if gw_i != 0:
        is_player_sold[gw] = [pulp.LpVariable(x+'__'+gw+'__sold',lowBound=0, upBound=1, cat='Integer') for x in gw_stats['id']]
        is_player_bought[gw] = [pulp.LpVariable(x+'__'+gw+'__bought',lowBound=0, upBound=1, cat='Integer') for x in gw_stats['id']]
        


In [65]:
# objective
model += sum([ (is_player_captain[gw][i] + is_player_in_team[gw][i])*points[gw][i]  for gw in game_weeks_to_consider for i in range(len(is_player_in_team[gw]))])

In [66]:
for gw_i, gw in enumerate(game_weeks_to_consider):
    # constraint 1 : cost
    model += sum((is_player_in_team[gw][i] + is_player_in_subs[gw][i])*costs[gw][i] for i in range(len(is_player_in_team[gw])))<=1000

    # constraint 2 - positional
    # GK
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='GK')) == 1
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='GK')) == 1

    # DEF
    model += sum((is_player_in_team[gw][i] + is_player_in_subs[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='DEF')) == 5
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='DEF')) <= 5
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='DEF')) >= 3
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='DEF')) >= 0
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='DEF')) <= 2

    # MID
    model += sum((is_player_in_team[gw][i] + is_player_in_subs[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='MID')) == 5
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='MID')) <= 5
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='MID')) >= 3
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='MID')) >= 0
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='MID')) <= 2

    # FWD
    model += sum((is_player_in_team[gw][i] + is_player_in_subs[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='FWD')) == 3
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='FWD')) <= 3
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])) if position[gw][i]=='FWD')) >= 1
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='FWD')) >= 0
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='FWD')) <= 2
    
    # constraint 3 - number of captain should be 1
    model += sum((is_player_captain[gw][i] for i in range(len(is_player_captain[gw])))) == 1

    # constraint 4 - number of team players should be 11
    model += sum((is_player_in_team[gw][i] for i in range(len(is_player_in_team[gw])))) == 11
    
    # constraint 5 - number of sub players should be 11
    model += sum((is_player_in_subs[gw][i] for i in range(len(is_player_in_subs[gw])))) == 4
    
    
    for i in range(len(is_player_in_team[gw])):
        # constraint 6 - player should be in either team or sub; not both
        model += (is_player_in_team[gw][i]+is_player_in_subs[gw][i])<=1
        
        # constraint 7 - captain should be in team
        model += (is_player_in_team[gw][i]-is_player_captain[gw][i]) >=0
        
    
    # constraint 8 - max 3 players from a team
    for tid in np.unique(team[gw]):
        model += sum(is_player_in_team[gw][i] + is_player_in_subs[gw][i] for i in range(len(is_player_in_team[gw])) if team[gw][i] == tid) <= 3
    
    if gw_i !=0:
        for i in range(len(is_player_in_team[gw])):
            # constraint 9 - same player should not be sold and bought
            model += (is_player_sold[gw][i] + is_player_bought[gw][i])<=1
            
            # constraint 10 - player sold should be in last week's team
            model += (is_player_in_team[game_weeks_to_consider[gw_i-1]][i] - is_player_sold[gw][i])>=0
            model += (is_player_in_subs[game_weeks_to_consider[gw_i-1]][i] - is_player_sold[gw][i])>=0
            
            # constraint 11 - player sold should not be in this week's team
            model += (is_player_in_team[gw][i] + is_player_sold[gw][i])<=1
            model += (is_player_in_subs[gw][i] + is_player_sold[gw][i])<=1
            
            # constraint 11* - player not sold should be in this week's team
            model += (is_player_in_team[gw][i] + is_player_in_subs[gw][i] + is_player_sold[gw][i] - is_player_in_team[game_weeks_to_consider[gw_i-1]][i] - is_player_in_subs[game_weeks_to_consider[gw_i-1]][i])>=0
            
            # constraint 12 - player bought should not be in last week's team
            model += (is_player_in_team[game_weeks_to_consider[gw_i-1]][i] + is_player_bought[gw][i])<=1
            model += (is_player_in_subs[game_weeks_to_consider[gw_i-1]][i] + is_player_bought[gw][i])<=1
            
            # constraint 13 - player bought should be in this week's team
            model += (is_player_in_team[gw][i] - is_player_bought[gw][i])>=0
            model += (is_player_in_subs[gw][i] - is_player_bought[gw][i])>=0
            
            # constraint 13* - player not bought should not be in this week's team
            model += (is_player_in_team[game_weeks_to_consider[gw_i-1]][i] + is_player_in_subs[game_weeks_to_consider[gw_i-1]][i] + is_player_bought[gw][i] - is_player_in_team[gw][i] - is_player_in_subs[gw][i])>=0
        
        # constraint 14 - number of players sold & bought should be equal in a week
        model += sum((is_player_sold[gw][i] - is_player_bought[gw][i]) for i in range(len(is_player_in_team[gw])))==0
        
        # constraint 15 - number transfers in 2 weeks should be less<=2
        if gw_i+1 <= len(game_weeks_to_consider)-1 :
            model += sum((is_player_sold[gw][i] + is_player_sold[game_weeks_to_consider[gw_i+1]][i]) for i in range(len(is_player_in_team[gw])))<=2

In [67]:
# pulp.listSolvers(onlyAvailable=True)
print(model.solve(pulp.PULP_CBC_CMD(msg=True)))
print(model.objective.value())

1
381.0


In [70]:
pulp.LpStatus[model.status]

'Optimal'

In [71]:
for gw_i, gw in enumerate(game_weeks_to_consider):
    print('\n{}'.format(gw))
    
    # constraint 1 : cost
    print('cost= {}'.format(sum((is_player_in_team[gw][i].value() + is_player_in_subs[gw][i].value())*costs[gw][i] for i in range(len(is_player_in_team[gw])))))

    # constraint 2 - positional
    # GK
    print('GK in team= {}'.format(sum((is_player_in_team[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='GK'))))
    print('GK in sub= {}'.format(sum((is_player_in_subs[gw][i].value() for i in range(len(is_player_in_subs[gw])) if position[gw][i]=='GK'))))
    

    # DEF
    print('DEF in team= {}'.format(sum((is_player_in_team[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='DEF')) ))
    print('DEF in sub= {}'.format(sum((is_player_in_subs[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='DEF')) ))
    
    
    # MID
    print('MID in team= {}'.format(sum((is_player_in_team[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='MID')) ))
    print('MID in sub= {}'.format(sum((is_player_in_subs[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='MID')) ))
    
    
    # FWD
    print('FWD in team= {}'.format(sum((is_player_in_team[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='FWD')) ))
    print('FWD in sub= {}'.format(sum((is_player_in_subs[gw][i].value() for i in range(len(is_player_in_team[gw])) if position[gw][i]=='FWD')) ))
    
    
    # constraint 3 - number of captain should be 1
    print('number of Captain= {}'.format(sum((is_player_captain[gw][i].value() for i in range(len(is_player_captain[gw])))) ))
    

    # constraint 4 - number of team players should be 11
    print('number of player in team= {}'.format(sum((is_player_in_team[gw][i].value() for i in range(len(is_player_in_team[gw])))) ))
    
    
    # constraint 5 - number of sub players should be 11
    print('number of player in sub= {}'.format(sum((is_player_in_subs[gw][i].value() for i in range(len(is_player_in_subs[gw])))) ))
    

    # constraint 6 - player should be in either team or sub; not both
    print('number of player in team & sub= {}'.format(sum((is_player_in_subs[gw][i].value()*is_player_in_team[gw][i].value() for i in range(len(is_player_in_subs[gw])))) ))

    
    # constraint 7 - captain should be in team
    print('captain in team?= {}'.format(sum((is_player_in_team[gw][i].value() for i in range(len(is_player_in_subs[gw])) if is_player_captain[gw][i].value()==1 )) ))
        
    
    # constraint 8 - max 3 players from a team
    print('max player from a team= {}'.format(max( sum(is_player_in_team[gw][i].value() + is_player_in_subs[gw][i].value() for i in range(len(is_player_in_team[gw])) if team[gw][i] == tid) for tid in np.unique(team[gw]))  ))
    
    if gw_i !=0:
        
        # constraint 9 - same player should not be sold and bought
        print('players sold & bought= {}'.format(sum((is_player_sold[gw][i].value()*is_player_bought[gw][i].value() for i in range(len(is_player_in_subs[gw])))) ))
        
        # constraint 10 - player sold should be in last week's team
        print('players sold & not in team= {}'.format(sum(( int(is_player_in_team[game_weeks_to_consider[gw_i-1]][i].value() < is_player_sold[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        print('players sold & not in sub= {}'.format(sum(( int(is_player_in_subs[game_weeks_to_consider[gw_i-1]][i].value() < is_player_sold[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        
        # constraint 11 - player sold should not be in this week's team
        print('players sold & in new team= {}'.format(sum(( (is_player_in_team[gw][i].value()*is_player_sold[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        print('players sold & in new sub= {}'.format(sum(( (is_player_in_subs[gw][i].value()*is_player_sold[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        
        # constraint 11* - player not sold should be in this week's team
        print('players not sold & not in new team= {}'.format(sum(( int((is_player_in_team[gw][i].value() + is_player_in_subs[gw][i].value()+is_player_sold[gw][i].value())<(is_player_in_team[game_weeks_to_consider[gw_i-1]][i].value() + is_player_in_subs[game_weeks_to_consider[gw_i-1]][i].value())) for i in range(len(is_player_in_subs[gw])))) ))
        
        
        # constraint 12 - player bought should not be in last week's team
        print('players bought already in team= {}'.format(sum(( (is_player_in_team[game_weeks_to_consider[gw_i-1]][i].value()*is_player_bought[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        print('players bought already in sub= {}'.format(sum(( (is_player_in_subs[game_weeks_to_consider[gw_i-1]][i].value()*is_player_bought[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        
            
        # constraint 13 - player bought should be in this week's team
        print('players bought & not in new team= {}'.format(sum(( int(is_player_in_team[gw][i].value() < is_player_bought[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        print('players bought & not in new sub= {}'.format(sum(( int(is_player_in_subs[gw][i].value() < is_player_bought[gw][i].value()) for i in range(len(is_player_in_subs[gw])))) ))
        
            
        # constraint 13* - player not bought should not be in this week's team
        print('players not bought & in new team= {}'.format(sum(( int((is_player_in_team[game_weeks_to_consider[gw_i-1]][i].value() + is_player_in_subs[game_weeks_to_consider[gw_i-1]][i].value() + is_player_bought[gw][i].value())<(is_player_in_team[gw][i].value() + is_player_in_subs[gw][i].value())) for i in range(len(is_player_in_subs[gw])))) ))
        
        # constraint 14 - number of players sold & bought should be equal in a week
        # constraint 15 - number transfers in 2 weeks should be less<=2
        print('players sold= {}'.format(sum(( is_player_sold[gw][i].value() for i in range(len(is_player_in_subs[gw])))) ))
        print('players bought= {}'.format(sum(( is_player_bought[gw][i].value() for i in range(len(is_player_in_subs[gw])))) ))
        


GW01
cost= 985.0
GK in team= 1.0
GK in sub= 1.0
DEF in team= 4.0
DEF in sub= 1.0
MID in team= 4.0
MID in sub= 1.0
FWD in team= 2.0
FWD in sub= 1.0
number of Captain= 1.0
number of player in team= 11.0
number of player in sub= 4.0
number of player in team & sub= 0.0
captain in team?= 1.0
max player from a team= 3.0

GW02
cost= 990.0
GK in team= 1.0
GK in sub= 1.0
DEF in team= 4.0
DEF in sub= 1.0
MID in team= 3.0
MID in sub= 2.0
FWD in team= 3.0
FWD in sub= 0.0
number of Captain= 1.0
number of player in team= 11.0
number of player in sub= 4.0
number of player in team & sub= 0.0
captain in team?= 1.0
max player from a team= 3.0
players sold & bought= 0.0
players sold & not in team= 0
players sold & not in sub= 0
players sold & in new team= 0.0
players sold & in new sub= 0.0
players not sold & not in new team= 0
players bought already in team= 0.0
players bought already in sub= 0.0
players bought & not in new team= 0
players bought & not in new sub= 0
players not bought & in new team= 0
p

In [72]:
is_player_in_team__result = np.array([[player.value() for player in is_player_in_team[gw]] for gw in game_weeks_to_consider]).T
is_player_in_subs__result = np.array([[player.value() for player in is_player_in_subs[gw]] for gw in game_weeks_to_consider]).T
is_player_captain__result = np.array([[player.value() for player in is_player_captain[gw]] for gw in game_weeks_to_consider]).T

In [75]:
result = is_player_in_team__result*2 + is_player_in_subs__result + is_player_captain__result
result = pd.DataFrame(result,columns=game_weeks_to_consider)
result.insert(0,'id', playerid[game_weeks_to_consider[0]])
result[result.sum(axis=1)>0].sort_values(by=game_weeks_to_consider,ascending=False).to_csv('result.csv',index=False)

In [76]:
result

,id,GW01,GW02,GW03
0,Aaron_Connolly__Brighton__FWD,0.0,0.0,0.0
1,Aaron_Cresswell__West_Ham__DEF,0.0,0.0,0.0
2,Aaron_Lennon__Burnley__MID,0.0,0.0,0.0
3,Aaron_Ramsdale__Arsenal__GK,0.0,0.0,0.0
4,Aaron_Ramsey__Aston_Villa__MID,0.0,0.0,0.0
...,...,...,...,...
576,Youri_Tielemans__Leicester__MID,0.0,0.0,0.0
577,Yves_Bissouma__Brighton__MID,0.0,0.0,0.0
578,Zack_Steffen__Man_City__GK,0.0,0.0,0.0
579,Álvaro_Fernández__Brentford__GK,0.0,0.0,0.0


In [44]:
# ## debug
# result = pd.DataFrame(np.concatenate([is_player_in_team__result,is_player_in_subs__result,is_player_captain__result],axis=1),
#                      columns = [gw+x for x in ['_team','_subs','_capt'] for gw in game_weeks_to_consider])
# result = result[sorted(result.columns)]
# result.insert(0,'id', playerid[game_weeks_to_consider[0]])
# result[result.sum(axis=1)>0].to_csv('result.csv',index=False)